# Deep Crossing

<img src="../img/deepcrossing.png" width="500">

- **Embedding 层**。将稀疏的类别型特征（one-hot）转换成稠密的 Embedding 向量
- **Stack 层**。把不同的 Embedding 特征和数值型特征拼接在 起，形成新的包含全部特征的特征向量
- **Multiple Residual Units 层**。通过多层残差网络对特征向量各个维度进行充分的交叉组合，使模型能够抓取到更多的非线性特征和组合特征的信息，进而使深度学习模型在表达能力上较传统机器习模型大为增强
- **Scoring 层**。用于拟合优化目标，对于 CTR 预估问题，Scoring 层往往使用逻辑回归模型

In [1]:
import pandas as pd
import numpy as np
import json
import torch
from torch import nn
from torch.utils.data.dataset import Dataset
from collections import OrderedDict

In [2]:
learning_rate = 0.00001
batch_size = 64
num_epochs = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
with open('../data/movielens/feature_map.json') as obj:
    feature_map = json.load(obj)
feature_map

{'dataset_id': 'movielens',
 'num_fields': 25,
 'feature_specs': {'movieId': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 935,
   'index': 0},
  'userId': {'source': 'user',
   'type': 'categorical',
   'vocab_size': 22540,
   'index': 1},
  'timestamp': {'source': 'user',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 3},
  'releaseYear': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 4},
  'movieGenre1': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 5},
  'movieGenre2': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 18,
   'index': 6},
  'movieGenre3': {'source': 'item',
   'type': 'categorical',
   'vocab_size': 15,
   'index': 7},
  'movieRatingCount': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 8},
  'movieAvgRating': {'source': 'item',
   'type': 'numerical',
   'vocab_size': 1,
   'index': 9},
  'movieRatingStddev': {'source': 'item',
   'type': 'nu

In [4]:
class MovielensDataset(Dataset):
    def __init__(self, url):
        self.df = pd.read_csv(url)
    
    def __getitem__(self, idx):
        x, y = self.df.iloc[idx, :-1].values.astype(np.float32), self.df.iloc[idx, -1].astype(np.float32)
        return x, y
    
    def __len__(self):
        return self.df.shape[0]

In [5]:
train_dataset = MovielensDataset('../data/movielens/data_for_train.csv')
test_dataset = MovielensDataset('../data/movielens/data_for_test.csv')

In [6]:
# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [7]:
class DeepCrossing(nn.Module):
    def __init__(self,
                 feature_map,
                 embedding_dim=10,
                 hidden_units=[256, 128, 64]):
        super(DeepCrossing, self).__init__()
        self.feature_map = feature_map
        self.embedding = nn.ModuleDict()
        for feature, feature_spec in feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                self.embedding[feature] = nn.Linear(1, embedding_dim, bias=False)
            elif feature_spec['type'] == 'categorical':
                padding_idx = feature_spec.get('padding_idx', None)
                self.embedding[feature] = nn.Embedding(feature_spec['vocab_size'],
                                                       embedding_dim,
                                                       padding_idx=padding_idx)
        input_dim = feature_map['num_fields'] * embedding_dim
        hidden_units = [input_dim] + hidden_units
        residual_layers = []
        for hidden_dim in hidden_units:
            residual_layers.append(ResidualBlock(input_dim,
                                                 hidden_dim))
        residual_layers.append(nn.Linear(input_dim, 1))
        self.residual_layer = nn.Sequential(*residual_layers)
        self.scoring = nn.Sigmoid()

    def forward(self, X):
        feature_emb_list = []
        for feature, feature_spec in self.feature_map['feature_specs'].items():
            if feature_spec['type'] == 'numerical':
                raw_feature = X[:, feature_spec['index']].float().view(-1, 1)
            elif feature_spec['type'] == 'categorical':
                raw_feature = X[:, feature_spec['index']].long()
            embedding_vec = self.embedding[feature](raw_feature)
            feature_emb_list.append(embedding_vec)
        feature_emb = torch.stack(feature_emb_list, dim=1)
        feature_residual = self.residual_layer(feature_emb.flatten(start_dim=1))
        y_pred = self.scoring(feature_residual).squeeze(1)
        return y_pred


class ResidualBlock(nn.Module):
    def __init__(self,
                 input_dim,
                 hidden_dim):
        super(ResidualBlock, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, input_dim)
        self.relu = nn.ReLU()

    def forward(self, X):
        residual = X
        out = self.fc1(X)
        out = self.relu(out)
        out = self.fc2(out)
        out += residual
        out = self.relu(out)
        return out

In [8]:
model = DeepCrossing(feature_map).to(device)
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (X, y) in enumerate(train_loader):
        X = X.to(device)
        y = y.to(device)

        # Forward pass
        output = model(X)
        loss = criterion(output, y)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 300 == 0:
            print("Epoch [{}/{}], Step [{}/{}] Loss: {:.4f}".format(
                epoch + 1, num_epochs, i + 1, total_step, loss.item()))

Epoch [1/5], Step [300/1388] Loss: 0.7103
Epoch [1/5], Step [600/1388] Loss: 0.6624
Epoch [1/5], Step [900/1388] Loss: 0.6904
Epoch [1/5], Step [1200/1388] Loss: 0.6088
Epoch [2/5], Step [300/1388] Loss: 0.6505
Epoch [2/5], Step [600/1388] Loss: 0.5854
Epoch [2/5], Step [900/1388] Loss: 0.5579
Epoch [2/5], Step [1200/1388] Loss: 0.6975
Epoch [3/5], Step [300/1388] Loss: 0.6292
Epoch [3/5], Step [600/1388] Loss: 0.7021
Epoch [3/5], Step [900/1388] Loss: 0.6143
Epoch [3/5], Step [1200/1388] Loss: 0.5633
Epoch [4/5], Step [300/1388] Loss: 0.5559
Epoch [4/5], Step [600/1388] Loss: 0.6446
Epoch [4/5], Step [900/1388] Loss: 0.4680
Epoch [4/5], Step [1200/1388] Loss: 0.5760
Epoch [5/5], Step [300/1388] Loss: 0.5210
Epoch [5/5], Step [600/1388] Loss: 0.5815
Epoch [5/5], Step [900/1388] Loss: 0.5284
Epoch [5/5], Step [1200/1388] Loss: 0.5842


In [9]:
# Test the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device).bool()
        output = model(X)
        y_pred = output > 0.5
        total += y.shape[0]
        correct += (y_pred == y).sum().item()

    print('Accuracy of the model on the test images: {:.2f} %'.format(
        100 * correct / total))

Accuracy of the model on the test images: 68.63 %
